In [100]:
import chess.pgn
import time
import sys
import math
import numpy as np

In [101]:
BOARD_SPACES = 64
ROWS = 8
COLS = 8

letter_lookup = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7
}
piece_lookup = {
    'r':1,
    'n':2,
    'b':3,
    'q':4,
    'k':5,
    'p':6,
    'R':-1,
    'N':-2,
    'B':-3,
    'Q':-4,
    'K':-5,
    'P':-6,
    '.':0
}

In [138]:
# Format of Dataset
# One chanel per piece
# One additional channel showing whose turn it is
# Referenced from https://int8.io/chess-position-evaluation-with-convolutional-neural-networks-in-julia/

dataset_x = []
dataset_y = []
posns = []

pgn = open("datasets/AllGames.pgn")
game = chess.pgn.read_game(pgn)
board = game.board()
black_or_white = True
count = 0

while game != None:
#     print(count)
    count += 1
    if count >= 100:
        break

    for move in game.mainline_moves():
        dp = []
        
        # Create a one-hot label for the move
        label = np.zeros(BOARD_SPACES*2)
        pos1 = str(move)[:2]
        pos1 = ROWS*letter_lookup[pos1[0]] + int(pos1[1])-1
        label[pos1] = 1
        pos2 = str(move)[2:]
        pos2 = ROWS*letter_lookup[pos2[0]] + int(pos2[1])-1
        label[(BOARD_SPACES) + pos2] = 1
        
        posns.append((pos1, pos2))
        
        # Turn the board into a vector
        strboard = str(board)
        board_data = np.zeros((7, 8, 8))
        idx = 0
        for char in strboard:
            if char in piece_lookup:
                if char != '.':
                    board_data[abs(piece_lookup[char])-1][int(idx/COLS)][idx%ROWS] \
                        = piece_lookup[char]/abs(piece_lookup[char])
                idx += 1
        # Fill the final channel with ones if white
        if black_or_white == True:
            board_data[6].fill(1)
                
        dataset_x.append(board_data)
        dataset_y.append(label)  
        black_or_white = not black_or_white
        board.push(move)
    game = chess.pgn.read_game(pgn)

dataset_x = np.stack(dataset_x)
dataset_y = np.stack(dataset_y)
dataset_x[0]

array([[[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [-1.,  0.,  0.,  0.,  0.,  0.,  0., -1.]],

       [[ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  0., -1.,  0.]],

       [[ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [139]:
inshape = dataset_x[0].shape
outshape = dataset_y[0].size
print(dataset_x.shape, outshape)

(7110, 7, 8, 8) 128


In [140]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adam, RMSprop
from keras.callbacks import ModelCheckpoint, TensorBoard

In [141]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(2, 2),
                 input_shape=inshape))
model.add(Activation('relu'))

model.add(Conv2D(128, kernel_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dense(1024))
model.add(Activation('relu'))

model.add(Dense(outshape, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

tensorboard = TensorBoard(log_dir='logs')
filepath = 'models/best_model.h5'
checkpoint = ModelCheckpoint(filepath, verbose=1,
                             save_best_only=True, mode='max')
callbacks = [tensorboard, checkpoint]

In [ ]:
model.fit(dataset_x, dataset_y, epochs=2000, batch_size=64, callbacks=callbacks)

Epoch 1/2000
7110/7110 [==============================] - 11s 2ms/step - loss: 9.5310 - acc: 0.0042
Epoch 2/2000
7110/7110 [==============================] - 11s 2ms/step - loss: 9.4851 - acc: 4.2194e-04
Epoch 3/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.4735 - acc: 0.0000e+00
Epoch 4/2000
7110/7110 [==============================] - 14s 2ms/step - loss: 9.4590 - acc: 0.0028
Epoch 5/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.4401 - acc: 0.0035
Epoch 6/2000
7110/7110 [==============================] - 11s 2ms/step - loss: 9.4160 - acc: 0.0013
Epoch 7/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.3861 - acc: 0.0018
Epoch 8/2000
7110/7110 [==============================] - 13s 2ms/step - loss: 9.3572 - acc: 0.0021
Epoch 9/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.3322 - acc: 0.0027
Epoch 10/2000
7110/7110 [==============================] - 13s 2ms/step - loss: 9.3065 - acc

7110/7110 [==============================] - 13s 2ms/step - loss: 9.1222 - acc: 0.0094
Epoch 83/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.1211 - acc: 0.0100
Epoch 84/2000
7110/7110 [==============================] - 11s 2ms/step - loss: 9.1218 - acc: 0.0110
Epoch 85/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.1222 - acc: 0.0111
Epoch 86/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.1231 - acc: 0.0117
Epoch 87/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.1213 - acc: 0.0110
Epoch 88/2000
7110/7110 [==============================] - 13s 2ms/step - loss: 9.1214 - acc: 0.0100
Epoch 89/2000
7110/7110 [==============================] - 13s 2ms/step - loss: 9.1207 - acc: 0.0118
Epoch 90/2000
7110/7110 [==============================] - 12s 2ms/step - loss: 9.1217 - acc: 0.0100
Epoch 91/2000
7110/7110 [==============================] - 15s 2ms/step - loss: 9.1202 - acc: 0.0101
Epoc

In [ ]:
score = model.evaluate(dataset_x, dataset_y, batch_size=128)
print(score)

In [93]:
output = model.predict(np.reshape(dataset_x[0], (1, 7, 8, 8)))
output = output[0]
val1 = np.amax(output[:64])
val2 = np.amax(output[64:])
idx1 = np.where(output == val1)[0]
idx2 = np.where(output == val2)[0]

print('Start: ', val1, idx1, 'End: ', val2, idx2)

(1, 128)
Start:  0.44285107 [25] End:  0.54137677 [91]


In [41]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print(input_shape)

# # convert class vectors to binary class matrices
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_test = keras.utils.to_categorical(y_test, num_classes)

# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),
#                  activation='relu',
#                  input_shape=input_shape))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(loss=keras.losses.categorical_crossentropy,
#               optimizer=keras.optimizers.Adadelta(),
#               metrics=['accuracy'])

# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))
# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

# # print(x_train[0])
# # print(y_train.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(28, 28, 1)
